In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# TODO:

- follow example to implement nearest neighbors model
- source and load iris data
- use implemented nn model to classify
- import nn model from sklearn and compare